In [1]:
import datetime
import logging
import pickle
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from absl import logging as aLogging
aLogging.set_verbosity(aLogging.ERROR)
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
tf.print(tf. __version__)
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from collections import namedtuple
import glob
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import math
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.get_logger().setLevel(tf.compat.v1.logging.FATAL)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
logging.getLogger('tensorflow').setLevel(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
%pdb on
import pdb

2.10.0
Num GPUs Available:  1
Automatic pdb calling has been turned ON


In [2]:
Stock = namedtuple('Stock', ['name', 'owned', 'dataframe', 'model'])

In [3]:
list_of_files = glob.glob('c:\\Users\\camar\\Downloads\\CAP5768\\data\\raw\\*.csv')
print('len(list_of_files)', len(list_of_files))

len(list_of_files) 10


In [4]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        return False
    def on_epoch_end(self, epoch, logs=None):
        return False
    def on_batch_begin(self, batch, logs=None):
        return False
    def on_batch_end(self, batch, logs=None):
        return False
    def on_predict_batch_end(self, batch, logs=None):
        return False
    def on_predict_batch_begin(self, batch, logs=None):
        return False
    def on_train_begin(self, batch, logs=None):
        return False
    def on_train_end(self, batch, logs=None):
        return False
    def on_train_batch_begin(self, batch, logs=None):
        return False
    def on_train_batch_end(self, batch, logs=None):
        return False
    def on_test_begin(self, batch, logs=None):
        return False
    def on_test_end(self, batch, logs=None):
        return False


In [5]:
look_back = 20
stocks = [None,None,None,None,None,None,None,None,None,None]
test = [1,2,3,4]
print(test[-1])

4


In [6]:
def predict(num_prediction, model, close_data):
    lb1 = look_back - 1
    prediction_list = close_data[-lb1:]
    
    for _ in range(num_prediction):
        x = prediction_list[-lb1:]
        x = x.reshape((1, lb1, 2))
        m = model.predict(x)
#         print('m', m)
        out = m[0][-1]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[lb1-1:]
#     print('prediction_list', prediction_list)
        
    return prediction_list

In [7]:
def printTransactions(m, k, d, stocks, dayIndex):
#     print('dayIndex', dayIndex)
#     print('k',k)
#     print('d',d)
    deltab = 0
    deltabi = -1
    # deltas = 999999999999999999
    # deltasi = -1
    lessThanZero = []
   
#     model = Sequential()
#     model.add(
#         LSTM(10,
#             activation='relu',
#             input_shape=(look_back,1))
#     )
#     model.add(
#         LSTM(10,
#             activation='relu',
#             input_shape=(look_back,1))
#     )
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse')

    num_epochs = 25

    for i in range(k):
        stock = stocks[i]
#         print('stock.model is None', stock.model is None, 'i', i, type(stock.model))
        if stock.model is None:
            model = Sequential()
            model.add(LSTM(units=50,return_sequences=True,activation='relu',input_shape=(look_back-1, 2)))
            model.add(Dropout(0.2))
            model.add(LSTM(units=100,return_sequences=True,activation='relu'))
            model.add(Dropout(0.2))
#             model.add(LSTM(units=200,return_sequences=True,activation='relu'))
#             model.add(Dropout(0.2))
#             model.add(LSTM(units=100,activation='relu'))
#             model.add(Dropout(0.2))
        #     model.add(LSTM(units=50,activation='relu'))
        #     model.add(Dropout(0.2))
            model.add(Dense(units=1))
            model.compile(optimizer='adam',loss='mean_squared_error') 
        else:
            model = stock.model
            
#         print('name', stock.name)
#         print('owned', stock.owned)
#         eprint('prices', stock.prices)
        df = stock.dataframe
#         close_data = df['Close'].values
        close_data = df[['Close', 'Volume']].values
        y_close_data = df['Close'].values
        y_close_data = y_close_data.reshape((-1,1))
#         close_data = df['Close'].values
#         print(f"close_data.shape {close_data.shape}")
#         close_data = close_data.reshape((-1,1))  
#         close_data = close_data.reshape((look_back + 1,2))  
#         print('len(close_data[:-1])', len(close_data[:-1]))
#         print('len(close_data[1:])', len(close_data[1:]))
#         print('close_data.shape[0] - 1', close_data.shape[0] - 1)
#         print(f"after reshape close_data.shape {close_data.shape}")
#         print(f"after reshape y_close_data.shape {y_close_data.shape}")
        train_generator = TimeseriesGenerator(close_data[:-1], y_close_data[1:], length=close_data.shape[0] - 2,
                                              batch_size=20)
#         dataset = tf.data.Dataset.from_tensor_slices(close_data[:-1], y_close_data[1:])        
#         model.fit(x=close_data[:-1], y=y_close_data[1:], batch_size=20, epochs=num_epochs,callbacks=[Callback()], verbose=0)
        model.fit(train_generator, epochs=num_epochs,callbacks=[Callback()], verbose=0)
#         ?model        
#         saver = model.fit.Saver()
#         print('type(saver)',type(saver))
        stock  = Stock(stock.name, stock.owned, stock.dataframe, model)
        stocks[i] = stock        
        pred1 = predict(1, model, close_data)[1]
#         print("pred1", pred1, 'stock.dataframe.iloc[-1].Close', stock.dataframe.iloc[-1].Close)
        d = (pred1 - stock.dataframe.iloc[-1].Close) / stock.dataframe.iloc[-1].Close
#         print(f"pred1 {pred1}  d {d}   stock.dataframe.tail(1).iloc[0].Close {stock.dataframe.tail(1).iloc[0].Close}")
        if d > deltab and d > 0:
            deltab = d
            deltabi = i
        # if d < deltas and stock.owned > 0 and d < 0:
        #     deltas = d
        #     deltasi = i
        if d <= 0 and stock.owned > 0:
            lessThanZero.append(stock)
#     print(f"deltab {deltab}  deltabi {deltabi}")
            
    # if deltasi == deltabi:
    #     deltabi = -1 # sell
            
            
    toSpend = round(m * min(1, 0.8 + deltab), 2)  
    sb = stocks[deltabi]
    sharesb = math.floor(toSpend / sb.dataframe.iloc[dayIndex].Close)
    count = 1 if deltabi >= 0 and sharesb > 0 else 0
#     print(f" toSpend: {toSpend} sharseb {sharesb}")
    count += len(lessThanZero)
#     print(count)
    # eprint(count)
    if deltabi >= 0 and sharesb > 0:
        stocks[deltabi] = Stock(sb.name, sb.owned + sharesb, sb.dataframe, sb.model)
        m -= sb.dataframe.iloc[-1].Close * sharesb
        # eprint(f"{sb.name} BUY {sharesb}")    
    for sIndex in range(len(lessThanZero)):
        ss = lessThanZero[sIndex]
#         print(f"ss: {ss}  ss.dataframe.shape {ss.dataframe.shape}")
        m += ss.owned * ss.dataframe.iloc[-1].Close
        stockIndex = [i for i,stock in enumerate(stocks) if stock.name == ss.name][0]
        stocks[stockIndex] = Stock(ss.name, 0, ss.dataframe, ss.model)
    return m


In [8]:
def runDay(data, dayIndex):
#     print(f"runDay start data[1].[2].shape {data[1][2].shape}")
#     lstocks = []
#     try:
#         with open('stocks.pkl', 'rb') as f:
#             stocks = pickle.load(f)
#     except:
#         pass
#         # for line in lines.split('\n'):
#         #     # eprint(f'line |{line}|')
#         #     if len(line) > 4:
#         #         sa = line.split()
#         #         # eprint(f'line sa {sa}')
#         #         stocks.append(Stock(sa[0], 0, list(map(float, sa[1:]))))

    # eprint(len(stocks))
    # eprint(stocks[0])

    m,k,d = list(map(float, data[0].split()))
    k = int(k)
    d = int(d)
    for di in range(k):
        sa = data[di+1]
        dataframe = sa[2]
#         print(f"runDay dataframe.shape {dataframe.shape}")
#         print('stock[di].model is None before', stocks[di].model is None, 'di', di, type(stocks[di].model))
        stocks[di] = Stock(sa[0], int(sa[1]), dataframe, stocks[di].model)
#         print('stock[di].model is None after', stocks[di].model is None, 'di', di, type(stocks[di].model))
#     print('printTransactions')
    m = printTransactions(m, k, d, stocks, dayIndex)
#     with open('stocks.pkl', 'wb') as f:
#         pickle.dump(stocks, f)
    return (m, stocks)

In [9]:
stockPrices = []
data = []
m = 10 # amount to spend each day
k = 10 # number of stocks
d = 0 # not used
data = []
data.append(f"{m} {k} {d}")
scaler = MinMaxScaler(feature_range=(0, 0.999))
for i, csv in enumerate(list_of_files):
    name = os.path.basename(csv)[0:-4]
    df = pd.read_csv(csv)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_axis(df['Date'], copy=False)
    df.drop(columns=['Open', 'High', 'Low'], inplace=True)
    #,Volume,Open,High,Low
    breakpoint()
    df.rename(columns={"Close/Last": "Close", "Date": "Date", "Volume": "Volume"}, inplace=True) 
    df['Close'] = pd.to_numeric(df['Close'].map(lambda x: x[1:])) / 1000
    df['Volume'] = pd.to_numeric(df['Volume'])
    dates = df.Date
    closes = df.Close
    scaled = scaler.fit_transform(df[['Volume']].values)
    df = pd.DataFrame(scaled, index=df.index, columns=['Volume'])
    df['Date'] = dates
    df['Close'] = closes
#     df.info()
    assert df.loc[df['Close'].idxmax()].Close <= 1
    assert df.loc[df['Volume'].idxmax()].Volume <= 1

    data.append((name,0,df.iloc[0:look_back + 1]))
    stock = Stock(name, 0, df, None)
#     print('stock.model is None', stock.model is None, 'i', i)
    stocks[i] = stock
    stockPrices.append(stock)
print('runDay')    
m, stocks = runDay(data, look_back)
data.clear()
total = m + sum([k.owned * k.dataframe.iloc[look_back].Close for k in stocks])
print(f"total: {total}  m: {m}") 
with open('stocks.pkl', 'wb') as f:
    pickle.dump(stocks, f)
with open('stockPrices.pkl', 'wb') as f:
    pickle.dump(stockPrices, f)    
with open('m.pkl', 'wb') as f:
    pickle.dump(m, f)        

runDay
1/1 [==============================] - 0s 174ms/step
total: 10.0  m: 0.02120000000000033


In [ ]:
stock = stockPrices[0]
stock.dataframe.loc[0:5].Close
stock.dataframe.iloc[-look_back:]
stock.dataframe.tail(1).iloc[0].Close
print(stock.dataframe.tail(1).iloc[0].Close)
print(stock.dataframe.iloc[-1].Close)


In [ ]:
with open('stocks.pkl', 'rb') as f:
    stocks = pickle.load(f)
with open('stockPrices.pkl', 'rb') as f:
    stockPrices = pickle.load(f)
with open('m.pkl', 'rb') as f:
    m = pickle.load(f)
data = []
k = 10
d = 0

In [10]:
firstPrices = [(s.dataframe.iloc[0].Close, s.name) for s in stockPrices] 
baseM = 10
baseStocks = []
for o in firstPrices:
    b = math.floor(1/o[0] + .35)
    baseM -= b * o[0]
    baseStocks.append(Stock(o[1], b, None, None))

In [11]:
 def getBaseValue():
    return baseM + \
        sum([[s.dataframe.iloc[-1].Close * stock.owned for s in stocks if s.name == stock.name][0] 
             for stock in baseStocks])

In [12]:
def checkForNegative(stocks):
    if len([k for k in stocks if k.owned * k.dataframe.iloc[-1].Close < 0]) > 0:
        print(f"Negative Stock! {[(k, k.owned, k.dataframe.iloc[-1]) for k in stocks if k.owned * k.dataframe.iloc[-1].Close < 0]}")

In [ ]:
startTime = datetime.datetime.now()
for dayIndex in range(look_back + 1, max(stockPrices, key=lambda k: k.dataframe.shape[0]).dataframe.shape[0]):        
    data.append(f"{m} {k} {d}")
    for stock in stocks:
        dataframe = [s.dataframe for s in stockPrices if s.name == stock.name][0]
#         print(f"dataframe.iloc[0:dayIndex].shape {dataframe.iloc[0:dayIndex].shape}")
        if dataframe.shape[0] > dayIndex + 2:
            data.append((stock.name, stock.owned, dataframe.iloc[dayIndex - look_back:dayIndex + 1]))
        else:
#             data.append((stock.name, stock.owned, dataframe.iloc[-(look_back + 1):]))
            continue
    if len(data) == k+1:  
        m, stocks = runDay(data, look_back)
        data.clear()
        checkForNegative(stocks)
#     if dayIndex % 20 == 0:
        total = m + sum([k.owned * k.dataframe.iloc[-1].Close for k in stocks])
        print(f"total: {total}  m: {m}  baseValue: {getBaseValue()}  dayIndex: {dayIndex}   {datetime.datetime.now()}  {(datetime.datetime.now() - startTime).seconds} seconds")  
    else:
        break
        
total = m + sum([k.owned * k.dataframe.iloc[-1].Close for k in stocks])
print(f"total: {total}  m: {m}  baseValue: {getBaseValue()}  {datetime.datetime.now()}  {(datetime.datetime.now() - startTime).seconds} seconds")  



1/1 [==============================] - 0s 26ms/step
total: 10.06004  m: 0.02120000000000033  baseValue: 11.607679999999998  dayIndex: 21   2022-11-04 16:23:59.556689  23 seconds
1/1 [==============================] - 0s 23ms/step
total: 10.02812  m: 0.02120000000000033  baseValue: 11.651789999999997  dayIndex: 22   2022-11-04 16:24:23.334497  47 seconds
1/1 [==============================] - 0s 22ms/step
total: 9.530320000000001  m: 0.02120000000000033  baseValue: 11.324139999999998  dayIndex: 23   2022-11-04 16:24:46.391016  70 seconds
1/1 [==============================] - 0s 22ms/step
total: 9.24684  m: 0.02120000000000033  baseValue: 11.144609999999998  dayIndex: 24   2022-11-04 16:25:10.273823  94 seconds
1/1 [==============================] - 0s 27ms/step
total: 9.3084  m: 0.02120000000000033  baseValue: 11.302689999999997  dayIndex: 25   2022-11-04 16:25:35.434598  119 seconds
1/1 [==============================] - 0s 27ms/step
total: 9.700560000000001  m: 0.02120000000000033  b

In [ ]:
stockPrices[0].dataframe.shape

In [ ]:
%%debug